In [19]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

CONTINENTS = ['Africa', 'Asia', 'Oceania', 'Europe', 'Northern America', 'South America', 'Antarctic Region']
TOP_PRODUCERS = ['United States of America',
 'India',
 'China',
 'Brazil',
 'Russian Federation',
 'Canada',
 'Indonesia',
 'Ethiopia',
 'Democratic Republic of the Congo',
 'Nigeria',
 'Sweden',
 'Chile',
 'Finland',
 'Germany',
 'France',
 'Uganda',
 'Ghana',
 'Mexico',
 'Myanmar',
 'Poland',
 'Viet Nam',
 'Pakistan',
 'Thailand',
 'Australia',
 'New Zealand',
 'Kenya',
 'United Republic of Tanzania',
 'Bangladesh',
 'Turkey']

In [20]:
FAOSTAT_country_data = pd.read_csv("/home/martin/Documents/school/EPFL/ws/ada/disappearing_forests/data/faostat/FAOSTAT_countries.csv", engine='python')

In [21]:
FAOSTAT_country_data = FAOSTAT_country_data[FAOSTAT_country_data.Country.isin(TOP_PRODUCERS) & FAOSTAT_country_data["Country Group"].isin(CONTINENTS)]

In [22]:
FAOSTAT_country_data = FAOSTAT_country_data[["Country", "Country Group"]]

In [23]:
# load datasets
forest_all = pd.read_csv("/home/martin/Documents/school/EPFL/ws/ada/disappearing_forests/data/faostat/Forestry_E_All_Data_cleared.csv", engine='python')

land = pd.read_csv("/home/martin/Documents/school/EPFL/ws/ada/disappearing_forests/data/faostat/Inputs_LandUse_E_All_Data_cleared.csv",engine='python')

In [24]:
forest_area = land[land.Item.isin(['Forestry']) & 
                   land.Area.isin(TOP_PRODUCERS) & 
                   land.Year.isin([1995, 2015])][["Area", "Year", "Unit", "Value", "Item"]]

In [25]:
forest_area.loc[forest_area.Year == 1995, 'Value'] = forest_area[forest_area.Year == 1995].Value.apply(lambda x: x*-1)


In [26]:
forest_area = forest_area.groupby(["Area"]).Value.sum().reset_index()

In [27]:
forest_area = forest_area.merge(FAOSTAT_country_data,left_on="Area", right_on="Country")

In [28]:
forest_area = forest_area.merge(country_area[["Area", "Value"]], left_on=["Area"], right_on=["Area"], suffixes=('_forest', '_country'))
 

In [29]:
forest_area["Decrease"] = forest_area["Value_forest"] / forest_area["Value_country"] 

In [30]:
forest_area = forest_area[["Area", "Decrease", "Country Group"]]

In [31]:
forest_area = forest_area.sort_values(by="Decrease")

In [64]:
forest_area.Decrease = forest_area.Decrease.apply(lambda x: x*100)

In [69]:
import plotly.express as px


fig = px.bar(forest_area, x='Decrease', y='Area', color='Country Group', orientation='h',
             labels={'Decrease':'Increase of forested area (% share in land area)', 'Area': 'Country', 'Country Group': "Continent"})

fig.update_layout(
    showlegend=False,
    yaxis_title=None,
    width=700,
    height=700,
) 

fig.show()

## Wood producers and deforestation

All of therese countries lead in the production of roundwood in the world, but the effects of wood logging on the forested area are not so obvios as one could expect. All of the european natios (light blue) were able not to decrease share of the forested areas even trough the fact that this group of 6 countries (Russia, Sweden, Finland, Germany, France, Poland) produces 13% of the world roundwood. Even in USA recorded increase, while beeing the world number one in 2015. The complete opposit trend can be observed in Africa (red) where in some of the countries, forested areas have decreased by more than 8% in last 20 years. 

In [70]:
# export the plot to html
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
import plotly.graph_objs as go
from IPython.core.display import display, HTML

plot(fig, filename = '3_production_deforestation.html')


'3_production_deforestation.html'